# Auto-Populate

**This is a draft, work in progress**

DataJoint’s superpower is declarative computation via dependencies: tables express what depends on what, and `populate()` orchestrates how/when to compute.

The `populate` mechanism in DataJoint is a cornerstone for automating data processing within pipelines. It enables users to execute computations for derived tables systematically, ensuring that all required data is processed, stored, and remains consistent with the upstream dependencies.

## Overview of `populate`

Derived tables in DataJoint are typically declared as `Computed` or `Imported` tables. These tables depend on upstream tables and are populated by executing computations that generate their content. The `populate` mechanism automates this process by:

1. Identifying unprocessed entries in the upstream dependencies.
2. Executing the computation logic defined in the `make` method of the table.
3. Inserting the resulting data into the derived table.

### Syntax

```python
<Table>.populate(safemode=True, reserve_jobs=False, display_progress=False)
```

### Parameters

1. **`safemode`** *(default: True)*:
   - Prompts for confirmation before populating the table.
   - Set to `False` to skip the confirmation prompt.
2. **`reserve_jobs`** *(default: False)*:
   - Enables reservation of jobs for distributed processing.
3. **`display_progress`** *(default: False)*:
   - Displays a progress bar for monitoring the population process.

## Declaring a Computed Table

To use the `populate` mechanism, define a derived table with a `make` method. The `make` method contains the logic for processing and populating the table.

### Example

```python
import datajoint as dj

schema = dj.Schema('example_schema')

@schema
class Animal(dj.Manual):
    definition = """
    animal_id: int  # Unique identifier for the animal
    ---
    species: varchar(64)  # Species of the animal
    age: int             # Age of the animal in years
    """

@schema
class AnimalSummary(dj.Computed):
    definition = """
    -> Animal
    ---
    age_in_months: int  # Age of the animal in months
    """

    def make(self, key):
        # Fetch the source data
        animal = (Animal & key).fetch1()
        # Compute derived data
        key['age_in_months'] = animal['age'] * 12
        # Insert the result into the table
        self.insert1(key)

# Insert example data
Animal.insert([
    {'animal_id': 1, 'species': 'Dog', 'age': 5},
    {'animal_id': 2, 'species': 'Cat', 'age': 3}
])

# Populate the AnimalSummary table
AnimalSummary.populate()
```

### Output
The `AnimalSummary` table will now contain:

```plaintext
animal_id | age_in_months
----------|---------------
    1     |      60
    2     |      36
```

## Using `populate` with Restrictions

The `populate` method can be restricted to process only specific entries.

### Example

```python
# Populate only entries for a specific animal
AnimalSummary.populate({'animal_id': 1})
```

## Distributed Processing with `reserve_jobs`

The `reserve_jobs` parameter facilitates distributed processing by reserving entries for parallel workers. This ensures that multiple workers do not process the same entry.

### Example

```python
AnimalSummary.populate(reserve_jobs=True)
```

## Best Practices

1. **Define Robust `make` Methods**:
   - Ensure that `make` handles all dependencies and edge cases.
2. **Use `populate` Incrementally**:
   - Test your `make` logic with specific keys before populating the entire table.
3. **Monitor Progress**:
   - Enable `display_progress` to track long-running population processes.
4. **Leverage Distributed Processing**:
   - Use `reserve_jobs` for large-scale pipelines to distribute the workload.
5. **Restrict When Necessary**:
   - Use restrictions to focus on specific entries during debugging or incremental processing.

## Summary

The `populate` mechanism in DataJoint automates the process of filling derived tables, ensuring consistent and efficient computation across your pipeline. By defining clear `make` methods and leveraging the flexibility of `populate`, you can streamline data processing workflows and maintain the integrity of your derived data.

